In [ ]:
from dotenv import load_dotenv
import os
import json
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai_like import OpenAILike
from llama_index.core.tools.types import ToolMetadata, BaseTool
from llama_index.tools.mcp import BasicMCPClient
def get_api_key():
    """Load the API key from the environment variable."""
    load_dotenv()
    return os.getenv("LITELLM_API_KEY")
API_KEY = get_api_key()
if not API_KEY:
    raise ValueError("API_KEY environment variable is not set. Please set it in your .env file.")

In [ ]:
# Connect to an MCP server using different transports
CONVEX_DIR_PATH = "/Users/tanujvasudeva/Documents/GitHub/kt-limitless/convex-app"
local_client = BasicMCPClient("npx", args=["-y", "convex@latest", "mcp", "start"])  # stdio

In [ ]:
async def get_server_tools():
    """Retrieve the tools from the MCP server."""
    valid_tool_names = ["status", "functionSpec", "run"]
    server_tools = await local_client.list_tools()
    tools = [tool for tool in server_tools.tools if tool.name in valid_tool_names]
    print(f"Tools available:")
    for idx, tool in enumerate(tools):
        print(f"{idx+1}. {tool.name}: {tool.description}\n")
    return tools

async def get_deploymentSelector():
    """Extract the deployment selector from the tool call result."""
    try:
        status_args = {"projectDir": CONVEX_DIR_PATH}
        response = await local_client.call_tool('status', status_args)
        json_string = response.content[0].text
        status = json.loads(json_string)
        if status.get("error"):
            print(f"Error: {status['error']}")
            return None
        return json.loads(json_string)["availableDeployments"][0]["deploymentSelector"]
    except:
        print("Error: MCP server ran into an error.")
        return None

async def get_backend_tools(deployment_selector: str):
    """Retrieve the function specification from the MCP server."""
    valid_backend_tools = ["lifelogs.js:searchMarkdown", "dashboard/sync.js:sync"]
    try:
        response = await local_client.call_tool('functionSpec', {"deploymentSelector": deployment_selector})
        json_string = response.content[0].text
        all_tools = json.loads(json_string)
    except:
        print("Error: MCP server is not running. Please start the server.")
        return None
    # Filter the tools based on the allow list
    allowed_tools = [
        tool for tool in all_tools
        if tool.get("identifier") in valid_backend_tools
    ]

    print("Allowed Tools:")
    for idx, tool in enumerate(allowed_tools):
        print(f"{idx+1}. {tool.get('identifier')}")
    return allowed_tools

In [ ]:
deployment_selector = await get_deploymentSelector()
if not deployment_selector:
    raise ValueError("Deployment selector could not be retrieved. Ensure the MCP server is running.")
backend_tools = await get_backend_tools(deployment_selector) or []
backend_tools

In [ ]:
MODEL_NAME = "nebius-qwen-4b"

In [ ]:
# Define a simple calculator tool
def multiply(a: float, b: float) -> float:
    """Useful for multiplying two numbers."""
    return a * b

def sync(sendNotification: bool = False) -> None:
    """Sync to the server."""
    print(f"Notification sent: {sendNotification}")


llm = OpenAILike(
    model=MODEL_NAME,
    api_base="http://localhost:4000/v1",
    is_function_calling_model=True,
    is_chat_model=True,
    api_key=API_KEY,
)

# # Create an agent workflow with our calculator tool
agent = FunctionAgent(
    tools=[],
    llm=llm,
    system_prompt="You are a helpful assistant that can multiply two numbers.",
)


# async def main():
#     # Run the agent
#     # response = await agent.run("What is 1234 * 4567?")
#     response = await agent.run("Run a quick sync.")
#     print(str(response))


# # Run the agent
# if __name__ == "__main__":
#     await main()

In [ ]:
response = await agent.run("What is 12323324 * 4567?")
print(str(response))